In [66]:
import numpy as np
from matplotlib import pyplot as plt
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [67]:
class Rect():
    def __init__(self, xs, ys, xe, ye):
        self.xs = xs # x start
        self.ys = ys # y start
        self.xe = xe # x end
        self.ye = ye # y end

    def __array__(self):
        return np.array([self.xs, self.ys, self.xe, self.ye])


In [174]:
aa = np.array([(1,2,3,4), (5,6,7,8)])
np.array(Rect(*aa[0]))

array([1, 2, 3, 4])

In [175]:
def describe_features(shape):
    rect2 = []
    rect3 = []
    rect4 = []
    height, width = shape
    for i in range(height+1):
        for j in range(width+1):
            for k in range(1, height+1):
                for l in range(1, width+1):
                    # @ 2 features
                    ij = np.array([i,j,i,j])
                    l1 = np.array([(0,l,k,2*l), (0,0,k,l)])
                    l2 = np.array([(0,0,k,l), (k,0,2*k,l)])
                    # Horizontal [-][+]
                    if i + k <= height and j + 2 * l <= width:
                        rect2.append((*(ij+l1[0]), *(ij+l1[1]), 0))

                    # Vertical #+
                        # -
                    if i + 2 * k <= height and j + l <= width:
                        rect2.append((*(ij+l2[0]), *(ij+l2[1]), 1))

                    # @ 3 features
                    l3 = np.array([(0,0,k,l), (0,l,k,2*l), (0,2*l,k,3*l)])
                    l4 = np.array([(0,0,k,l), (k,0,2*k,l), (2*k,0,3*k,l)])
                    # Horizontal [-][+][-]
                    if i + k <= height and j + 3 * l <= width:
                        rect3.append((*(ij+l3[0]), *(ij+l3[1]), *(ij+l3[2]), 2))

                    # Vertical #-
                        # +
                        # -
                    if i + 3 * k <= height and j + l <= width:
                        rect3.append((*(ij+l4[0]), *(ij+l4[1]), *(ij+l4[2]), 3))

                    # @ 4 features
                    l5 = np.array([(0,0,k,l), (0,l,k,2*l),(k, 0, 2*k, l), (k, l, 2*k, 2*l)])
                    # [-][+]
                    # [+][-]
                    if i + 2 * k <= height and j + 2 * l <= width:
                        rect4.append((*(ij+l5[0]), *(ij+l5[1]), *(ij+l5[2]), *(ij+l5[3]), 4))
    return np.array(rect2), np.array(rect3), np.array(rect4)


In [176]:
%time
f2, f3, f4= describe_features((19,19))
np.save('f2.npy', f2)
np.save('f3.npy', f3)
np.save('f4.npy', f4)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [177]:
print(f2.shape)
print(f3.shape)
print(f4.shape)
print(f2.shape[0] + f3.shape[0] + f4.shape[0])
print(c)

(86400, 9)
(55200, 13)
(20736, 17)
162336
360000


# Comparison

In [178]:
fkak = np.load('/home/akram/CMP4/GP/Try/ft.npy')
# fkak = np.load('/home/akram/CMP4/GP/Try/f5.npy')

In [179]:
fkak.shape

(162336, 5)

# feature 2 - Horizontal

In [180]:
f22 = f2[f2[:, -1] == 0]
f22 = np.stack((np.zeros(len(f22)), f22[:,5],f22[:,0], f22[:,3] - f22[:,5], f22[:,2] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == 0]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), ' true out of ', f22.size)
print(f22.shape)
print(fk2.shape)


216000  true out of  216000
(43200, 5)
(43200, 5)


# Feature 2 - Vertical

In [181]:
f22 = f2[f2[:, -1] == 1]
f22 = np.stack((np.ones(len(f22)), f22[:,1],f22[:,0], f22[:,3] - f22[:,1], f22[:,6] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == 1]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), ' true out of ', f22.size)
print(f22.shape)
print(fk2.shape)


216000  true out of  216000
(43200, 5)
(43200, 5)


# Feature 3 - Horizontal

In [182]:
idx = 2
cur_f = f3
f22 = cur_f[cur_f[:, -1] == idx]
f22 = np.stack((np.repeat(idx,len(f22)), f22[:,1],f22[:,0], f22[:,11] - f22[:,1], f22[:,10] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == idx]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), ' true out of ', f22.size)
print(f22.shape)
print(fk2.shape)


138000  true out of  138000
(27600, 5)
(27600, 5)


# Feature 3 - Vertical

In [183]:
idx = 3
cur_f = f3
f22 = cur_f[cur_f[:, -1] == idx]
f22 = np.stack((np.repeat(idx,len(f22)), f22[:,1],f22[:,0], f22[:,3] - f22[:,1], f22[:,10] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == idx]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), ' true out of ', f22.size)
print(f22.shape)
print(fk2.shape)


138000  true out of  138000
(27600, 5)
(27600, 5)


# Feature 4

In [184]:
idx = 4
cur_f = f4
f22 = cur_f[cur_f[:, -1] == idx]
f22 = np.stack((np.repeat(idx,len(f22)), f22[:,1],f22[:,0], f22[:,7] - f22[:,1], f22[:,10] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == idx]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), ' true out of ', f22.size)
print(f22.shape)
print(fk2.shape)

103680  true out of  103680
(20736, 5)
(20736, 5)
